# 1. Introduction

<img src="intro1.png" alt="score" width="500" height="200" 
style="float:left;">


So the key technology for handling datasets that are this big is called big data. Unfortunately, these days, big data is a buzzword and everyone's got their own little definition. **I like to think of big data technology as any technology that requires distributed computing.** 

Case 1: if you have a database, but that database uses what's called sharding, that means user IDs, 1 to 1000 will be on one machine, whereas user IDs, 1001 to 2000 will be on another machine and so on.

Case 2: You can even sharding the files themselves. So if you have a humongous log file of all the events that are happening on your website and say the file is one terabyte large, then you might store pieces of that file across different machines around the world.

Case 3: Finally, you can distribute compute itself. So imagine you have to do a very long for loop, just like we have to do in matrix factorization. So we have to write for i in range N but N is 1.8 billion. So that's not going to happen on a single machine in any reasonable amount of time. Instead, the first 1000 users will be processed on one machine. The second 1000 users will be processed on another machine and so forth. This is what we're mostly interested in, at least for this course. Our actual data file is small enough to fit on our local machine, but the algorithm that we use to process it in particular matrix factorization, can be sped up by distributing the compute across multiple machines.



<img src="intro2.png" alt="score" width="500" height="200" style="float:left;">
<img src="intro3.png" alt="score" width="400" height="200" style="float:left;">

# Section Outline
<img src="intro4.png" alt="score" width="400" height="200" style="float:left;">

# 2. Spark install on local MacOS
Detailed steps are not included here.

After install, termal "spark-shell", test it with some code, and ":q" to exit.

What we'll actually be using is pyspark. It's still spark, but we can write our code in Python. This is the interface we'll be using for writing our matrix factorization code. So if you've gotten this far, then you're ready to head over to that lecture. In terminal, type "pyspark".

Basically, you need to install:
- Homebrew
- Xcode
- Java
- Scala
- apache-spark


# 3. MF in Spark on local machine
## 3.1 
Meant to be pasted into console, not in jupyter notebook.

Check the **spark.py** script.

To run it, initiate the "pyspark" command in terminal, then past the code to terminal.

## 3.2 
A way to run a python script on Spark.

Check the **spark2.py** script.

This is, by the way, a stepping stone for the next step, which is running the job on a cluster using Amazon Web services.

To run the code, in terminal :
- do not initiate the pyspark command
- spark-submit --master spark://localhost:7077 spark2.py

# 4. MF in Spark on AWS/EC2

Check out the video: https://www.udemy.com/course/recommender-systems/learn/lecture/11798760#overview

# 5. Use Spark to make predictions in the real world

<img src="intro5.png" alt="score" width="500" height="200" style="float:left;">

<img src="intro6.png" alt="score" width="500" height="200" style="float:left;">

So there's a discrepancy between the timescales of these two kinds of tasks.

Most likely your API is pulling data from some database, maybe Postgres or MySQL. So then your job in the Spark script is to save your output to this database. We won't discuss the code to do that, but you shouldn't have too much trouble finding a one liner through Google.

<img src="intro7.png" alt="score" width="500" height="200" style="float:left;">
